In [ ]:
1 + 1

In [ ]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

In [ ]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [ ]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules
using PyCall
pd = pyimport("pandas")

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("network.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "nbody.jl"))
    includef(srcdir("scenarios", "bounce.jl"))
    includef(srcdir("scenarios", "mat.jl"))
end

In [ ]:
function report_synth(dataset, method; verbose=false, priordim=true, priortrans=true)
    df = pd.DataFrame(
        columns=("nRMSE", "#Scenes", "Seed", "Seed (Data)")
    )

    count = 1
    @showprogress for ntrains in 1:10, seed in 1:3, shuffleseed in 1:5
        hps = (ntrains=ntrains, seed=seed, shuffleseed=shuffleseed)
        fn = "monly"
        if method in ("bsp", "zero")
            hps = (hps..., priordim=priordim, priortrans=priortrans)
        else
            fn = "$fn-$method"
        end

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 20; shuffleseed=hps.shuffleseed, istrain=false)

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "$fn.jld2"))
        catch
            println("cannot find $fn")
            println("seed $(hps.seed) unfinished")
            continue
        end

        local ScenarioModel, force, nm, mlike
        if method in ("bsp", "zero")
            @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, force = res
            if method == "zero"
                force = ZeroForce()
            end
        else
            @unpack ScenarioModel, latentname, nm, mlike, ptrace = res
        end

        verbose && println("count: $count")
        
        verbose && if method == "bsp"
            println(BayesianSymbolic.get_executable(force.tree, force.grammar))
            println(force.constant)
        end
        if method in ("bsp", "zero")
            mmethod = force
        else
            mmethod = nm
        end
        
        evalres = evalm(ScenarioModel, scenarios, attributes, mmethod, mlike)
        evalres_test = evalm(ScenarioModel, scenarios_test, attributes_test, mmethod, mlike)

        # Add to data frame
        df.loc[count] = [evalres_test.normrmse_oracle, ntrains, seed, shuffleseed]
        count = count + 1
        verbose && println()
    end
    
    return df
end

## Figure 4

In [ ]:
for dataset in ("nbody", "bounce", "mat")
    for method in ("bsp", "zero", "ogn", "in", "mlpforce", "mlpdynamics")
        df = report_synth("synth/$dataset", method)
        df.to_csv(projectdir("paper", "raw", "efficiency-$dataset-$method.csv"), header=false)
    end
end

## Figure 5

In [ ]:
for dataset in ("nbody", "bounce", "mat")
    for priordim in (false, true), priortrans in (false, true)
        df = report_synth("synth/$dataset", "bsp"; priordim=priordim, priortrans=priortrans)
        df.to_csv(projectdir("paper", "raw", "efficiency-$dataset-g$(Int(priordim))$(Int(priortrans)).csv"), header=false)
    end
end

## Number of expressions in search space

In [ ]:
c00 = count_expressions(G_BSP_00, 5, :Force)
c01 = count_expressions(G_BSP_01, 5, :Force)
c10 = count_expressions(G_BSP_10, 5, :Force)
c11 = count_expressions(G_BSP_11, 5, :Force)
@info "Depth 5" c00 c01 c10 c11

# For depth 6, only c11 is tractable to compute.
c11 = count_expressions(G_BSP_11, 6, :Force)
@info "Depth 6" c11